In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board

import numpy as np

In [6]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val")

In [7]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=1)

In [8]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

2022-03-27 19:17:28.650999: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 19:17:28.654141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 19:17:28.654464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 19:17:28.654957: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model = tf.keras.models.load_model("production/")

2022-03-27 18:37:47.672673: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-03-27 18:37:47.696176: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-03-27 18:37:47.730876: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


In [10]:
#2890/2890 [==============================] - 109s 38ms/step - loss: 0.2221 - mae: 0.4492 - val_loss: 0.2205 - val_mae: 0.4457
#2890/2890 [==============================] - 100s 35ms/step - loss: 0.2091 - mae: 0.4277 - val_loss: 0.2259 - val_mae: 0.4423
model.fit(training_generator, validation_data=val_generator, epochs=1)

2890/2890 [==============================] - 100s 35ms/step - loss: 0.2091 - mae: 0.4277 - val_loss: 0.2259 - val_mae: 0.4423


In [6]:
indicies = get_halfkp_indeicies(Board("3R4/2P2kp1/b1r2p1p/1p5P/p4B2/P4P2/6PK/8 b - - 77 81"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

[0.5957211]


In [7]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[0.05389988] 1.0 r5k1/5p2/4b1p1/q3P3/8/6Pp/P3RQ1P/4R1K1 w - - 1 44
[0.22341466] 1.0 r1b1k2r/2qp1ppp/p4n2/1p2b3/4P3/1B6/PP3PPP/RNBQ1RK1 w kq - 0 12
[0.07042468] 1.0 r2qk2r/pp1bbpp1/2npp2p/8/3NN2B/8/PPPQ1PPP/1K1R1B1R b kq - 0 11
[0.3372172] 1.0 8/8/5n2/8/P6k/4NK2/8/8 w - - 0 57
[-0.10945517] 0.0 R7/5pk1/1p3p2/7p/P6P/5PK1/1r4P1/8 b - - 2 37
[-0.23685908] -1.0 8/pp4pk/2q4p/8/5PK1/6QP/PP6/8 b - - 5 50
[-0.0353601] -1.0 2kr2r1/ppp1q3/4pp2/3p1npp/2PP1P2/PP2P3/1B1Q2PP/R4RK1 b - - 2 19
[0.11502671] -1.0 rnb1r1k1/1p3p1p/p2N2p1/2pP4/P3Pb1q/2N2Q2/1P3PBP/R4RK1 w - - 0 18
[0.00280166] -1.0 r2q1rk1/p3ppbp/2Bp1np1/8/2P5/2N3Pb/PP2PP1P/R1BQ1RK1 w - - 1 11
[-0.0111109] -1.0 rnbr2k1/pp3ppp/8/1R1pP3/2p2P2/q4B2/P1PQ1PPP/5RK1 b - - 2 16
[-0.01358819] -1.0 r1bqk2r/ppp1ppb1/2np1npp/8/3PP2B/2N2P1P/PPPQ2P1/R3KBNR b KQkq - 0 9
[-0.11784208] 1.0 2r5/4R3/pk1Pp3/8/7p/1P6/P1P2r2/1K2R3 w - - 2 32
[-0.2560897] -1.0 8/2r4p/7K/3k3P/8/P7/2p5/2R5 b - - 0 39
[0.07278204] 1.0 rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQk

In [8]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-03-27 19:05:01.600799: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1648422301/assets
